## Lecture du CSV Stations + combinaisons avec le json crée pour le front

In [80]:
import pandas as pd
import json

# Read the csv file
df = pd.read_csv('../CSV/stations.csv', sep=';')

# Read the json file
with open('../testaffichagestation/public/svgLines/stations.json', encoding="UTF-8") as json_file:
    jsonfile = json.load(json_file)

# Convert json file "stations" key to dataframe
df2 = pd.DataFrame(jsonfile['stations'])

# la colonne id peut être un string ou un int (oupsi)
df2['id'] = df2['id'].astype(int)

# convertir position en position_x et position_y (json)
df2[['position_x', 'position_y']] = df2['position'].apply(pd.Series)

# Supprimer la colonne position
df2 = df2.drop('position', axis=1)

df_meg = pd.merge(df, df2, on='id', how='left')

print("(len(df) = ", len(df))
print("(len(df2) = ", len(df2))
print("(len(df_meg) = ", len(df_meg))

display(df_meg)

display(df[df['id'] == 5])
display(df2[df2['id'] == 5])
display(df_meg[df_meg['id'] == 5])

# liste des id de df qui ne sont pas dans df_meg
print("liste des id de df qui ne sont pas dans df_meg")
display(df[~df['id'].isin(df_meg['id'])])

display(df_meg.iloc[145])

(len(df) =  422
(len(df2) =  400
(len(df_meg) =  422


,id,ligne,terminus,nom,idfmId,idName,displayName,line,displayType,position_x,position_y
0,0,12,0,Abbesses,,station-Abbesses,Abbesses,M12,normal,2188.0000,1271.0000
1,1,2,0,Alexandre_Dumas,,station-AlexandreDumas,Alexandre Dumas,M2,normal,3425.9395,2382.5195
2,2,9,0,Alma_Marceau,,station-AlmaMarceau,Alma - Marceau,M9,normal,1242.0000,2271.0000
3,3,4,0,Alésia,,station-Alesia,Alésia,M4,normal,2195.6063,3521.7991
4,4,3,0,Anatole_France,,station-AnatoleFrance,Anatole France,M3,normal,1184.7131,1148.5858
...,...,...,...,...,...,...,...,...,...,...,...
417,453,RERE,0,Noisy_le_Sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,450,RERE,0,Pantin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,456,RERE,0,Rosa_parks,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,454,RERE,1,Chelles_Gournay,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,id,ligne,terminus,nom
5,5,2,0,Anvers


,id,idfmId,idName,displayName,line,displayType,position_x,position_y
70,5,,station-Anvers,,M2,normal,2351.6374,1361.2755


,id,ligne,terminus,nom,idfmId,idName,displayName,line,displayType,position_x,position_y
5,5,2,0,Anvers,,station-Anvers,,M2,normal,2351.6374,1361.2755


liste des id de df qui ne sont pas dans df_meg


,id,ligne,terminus,nom


id                                   145
ligne                                 10
terminus                               0
nom                                Javel
idfmId                                  
idName         station-JavelAndreCitroen
displayName        Javel - André Citroën
line                                 M10
displayType                       normal
position_x                         975.0
position_y                        3027.0
Name: 145, dtype: object

## Lecture relations

In [81]:
import pandas as pd

# Read the csv file
df_rel = pd.read_csv('../CSV/relations.csv', sep=';')

display(df_rel)

print(len(df_rel))

# print rel where id1 or id2 is 436
display(df_rel[(df_rel['id1'] == 436) | (df_rel['id2'] == 436)])


,id1,id2,temps
0,0,238,41
1,0,159,46
2,1,12,36
3,1,235,44
4,2,110,69
...,...,...,...
610,450,453,217
611,456,450,198
612,414,456,215
613,397,449,130


615


,id1,id2,temps
597,399,436,1821


## Insertions des stations et des relations dans la BDD

In [82]:
# upload in BD
%load_ext dotenv 
%dotenv -o -v .env

from mysql.connector import connect, Error
import os

try:
    db = connect(
        host = os.getenv('db_host'),
        user = os.getenv('db_username'),
        password = os.getenv('db_password'),
        database = os.getenv('db_name')
    )
    print(db)
    cursor = db.cursor()
    # show tables
    cursor.execute("SHOW TABLES")
    for table in cursor:
        print(table)
except Error as e:
    print("Error while connecting to MySQL", e)

# Df columns : id	ligne	terminus	nom	idfmId	idName	displayName	line	displayType	position_x	position_y

# on garde
# id : id de la station
# terminus : boolean
# nom : nom de la station (dans le fichier csv de base)
# idfmId : id de la station selon IDFM (pas remplis pour le moment mais peut être utile)
# idName : id à afficher en html
# displayName : nom à afficher en html
# line : ligne de la station
# displayType : displayType pour react
# position_x : position x de la station
# position_y : position y de la station

# supprimer table si existe

print("Suppression de la table stations si elle existe")

try:
    cursor.execute("DROP TABLE relations")
except Error as e:
    print("Error while dropping table", e)

try:
    cursor.execute("DROP TABLE stations")
except Error as e:
    print("Error while dropping table", e)

print("Suppression de la table relations si elle existe")

print("Création des tables stations et relations")
# Create table stations
try:
    query = """
    CREATE TABLE stations (
        id INT PRIMARY KEY, 
        terminus BOOLEAN,
        nom VARCHAR(100),
        idfmId VARCHAR(50),
        idName VARCHAR(255) ,
        displayName VARCHAR(255),
        line VARCHAR(20),
        displayType VARCHAR(50),
        position_x INT,
        position_y INT,
        virtual BOOLEAN DEFAULT FALSE
    )
    """
    cursor.execute(query)
except Error as e:
    print("Error while creating table stations", e)

# create table relations
try:
    query = """
    CREATE TABLE relations (
        id1 INT,
        id2 INT,
        temps INT,
        PRIMARY KEY (id1, id2),
        FOREIGN KEY (id1) REFERENCES stations(id) ON DELETE CASCADE,
        FOREIGN KEY (id2) REFERENCES stations(id) ON DELETE CASCADE
    )
    """
    cursor.execute(query)
except Error as e:
    print("Error while creating table rel", e)


print("Insertion des données dans les tables stations et relations")
# Insert stations
try:
    curent_row = 0
    query = """
    INSERT INTO stations (id, terminus, nom, idfmId, idName, displayName, line, displayType, position_x, position_y) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    for index, row in df_meg.iterrows():
        if(row['id'] == 13):
            print("index 13")
            print(row)
        curent_row = index
        # on utilise line si pas null sinon on utilise ligne
        line = row['line'] if row['line'] != '' and not pd.isnull(row['line']) else row['ligne']

        cursor.execute(query, (row['id'], row['terminus'], row['nom'], row['idfmId'], row['idName'], row['displayName'], line, row['displayType'], row['position_x'], row['position_y']))
    db.commit()
except Error as e:
    print("Error while inserting rows", e)
    print("Current row : ", df_meg.iloc[curent_row])

# Insert relations
try:
    query = """
    INSERT INTO relations (id1, id2, temps) 
    VALUES (%s, %s, %s)
    """
    current_row = 0
    for index, row in df_rel.iterrows():
        current_row = index

        # vérifier id1 et id2 existent dans df_meg
        if(not (row['id1'] in df_meg['id'].values and row['id2'] in df_meg['id'].values)):
            print("id1 ou id2 n'existent pas dans df_meg")
            print("id1 = ", row['id1'])
            print("id2 = ", row['id2'])
            continue

        cursor.execute(query, (int(row['id1']), int(row['id2']), int(row['temps'])))
    db.commit()
except Error as e:
    print("Error while inserting rows", e)
    print("Current row : ", df_rel.iloc[current_row])

print("Fin de l'insertion des données")

query = "UPDATE stations SET displayName = nom WHERE displayName = '';"
cursor.execute(query)
db.commit()




The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
('relations',)
('stations',)
Suppression de la table stations si elle existe
Suppression de la table relations si elle existe
Création des tables stations et relations
Insertion des données dans les tables stations et relations


index 13
id                                     13
ligne                                   2
terminus                                0
nom                   Barbès_Rochechouart
idfmId                                   
idName         station-BarbesRochechouart
displayName         Barbès - Rochechouart
line                                   M2
displayType                          none
position_x                      2544.4861
position_y                      1361.2754
Name: 13, dtype: object
id1 ou id2 n'existent pas dans df_meg
id1 =  407
id2 =  396
id1 ou id2 n'existent pas dans df_meg
id1 =  407
id2 =  124
id1 ou id2 n'existent pas dans df_meg
id1 =  407
id2 =  125
id1 ou id2 n'existent pas dans df_meg
id1 =  407
id2 =  151
id1 ou id2 n'existent pas dans df_meg
id1 =  408
id2 =  402
id1 ou id2 n'existent pas dans df_meg
id1 =  408
id2 =  395
id1 ou id2 n'existent pas dans df_meg
id1 =  408
id2 =  67
id1 ou id2 n'existent pas dans df_meg
id1 =  408
id2 =  68
id1 ou id2 n'existent pas d